In [ ]:
%python
import datetime
import sys

#SDFSDFSsdfsdfsdf 123123
#Define variables

#
# add by Jason for testing 2022/11/10
# 
start_time = datetime.datetime.now();

tgt_objectname = 'dwd_ms.ln_quotation';
src_objectname1 = 'prep_ms_row.lbp_quotation';
src_objectname2 = 'prep_ms_row.lbp_quoteline';

#主表标识
main_flg = {src_objectname1:1,src_objectname2:1}

main_tab_no_data_num = []
all_tab_no_data_num = []

def get_source_table_version_info(tgt_objectname, src_objectname):
    
    last_commit_time = datetime.datetime.now();
      
    version_str = spark.sql(f"""
                             select max(data_watermark_value) as version from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                         """).collect()[0]['version'];
    
    #print("version_str" + str(i) + version_s
    if version_str == None:
        version_str = 0
    else:
        #Get last end_time in edw_loading_control list
        last_commit_time = spark.sql(f"""
                                          select commit_time from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                                      """).collect()[0]['commit_time']
        
        #Get this commit_timestamp in change_log list
        initialize_commit_time = spark.sql(f"""
                                          select _commit_timestamp as commit_timestamp from table_changes('{src_objectname}',0)
                                          where _commit_version < int({version_str})
                                      """).collect()[0]['commit_timestamp']
    
        new_version = spark.sql(f"""
                                     select min(_commit_version) as new_version from table_changes('{src_objectname}',0)
                                     where _commit_version > int({version_str})
                                       and _change_type <> 'update_preimage'
                                     ;
                                 """).collect()[0]['new_version'];
        
    max_version_str = spark.sql(f"""
                                 select max(_commit_version) as max_version from table_changes('{src_objectname}',0);
                             """).collect()[0]['max_version'];
    
    if max_version_str == None:
        max_version = 0
    else:
        max_version = int(max_version_str)
    
    version = 0;
    
    if version_str == 0:
        print(src_objectname + ' : No data in the edw_loading_control list!')
        scene_id = 1
        version  = 0
    else:
        print(src_objectname + ' : Have the data in the edw_loading_control list!')
        #Determines whether the source table initializes data
        if initialize_commit_time > last_commit_time and main_flg.get(src_objectname) == 1:
            print(src_objectname + ' : The source table was drop, full capacity run number')
            scene_id = 2
            version = 0
            main_tab_no_data_num.append(1)
        elif initialize_commit_time > last_commit_time and main_flg.get(src_objectname) == 0:
            print(src_objectname + ' : The source table was drop, full capacity run number')
            scene_id = 3
            version = 0
        else:
            if new_version == None :
                print(src_objectname + ' : No new data')
                scene_id = 4
                version = int(version_str)
                all_tab_no_data_num.append(1)
            else:
                scene_id = 5
                version = int(new_version)
    return scene_id, version, max_version, last_commit_time

def update_control_table(tgt_objectname, src_objectname, version, max_version):
    #Get this max commit time in change_log list
    max_commit_time = spark.sql(f"""
                                     select max(_commit_timestamp) as commit_time from table_changes('{src_objectname}',{version},{max_version})
                                 """
                                 ).collect()[0]['commit_time'];
    
    
    spark.sql(f"""
                   merge into dwd_ms.edw_loading_control as target
                   using (
                          select 
                              '{tgt_objectname}'   as tgt_objectname
                              ,'{src_objectname}'  as src_objectname
                              ,'{start_time}'      as start_time
                              ,'{end_time}'        as end_time
                              ,'{max_version}'     as data_watermark_value
                              ,'{max_commit_time}' as commit_time
                          ) as source
                   on  target.tgt_objectname = source.tgt_objectname
                   and target.src_objectname = source.src_objectname
                   when matched then
                     update set target.start_time            = source.start_time
                                ,target.end_time             = source.end_time
                                ,target.data_watermark_value = source.data_watermark_value
                                ,target.commit_time          = source.commit_time
                   when not matched
                     then insert(
                                 target.tgt_objectname
                                 ,target.src_objectname
                                 ,target.start_time
                                 ,target.end_time
                                 ,target.data_watermark_type
                                 ,target.data_watermark_value
                                 ,target.process_status
                                 ,target.error_number
                                 ,target.error_message
                                 ,target.commit_time
                                 )
                           values(
                                 source.tgt_objectname
                                 ,source.src_objectname
                                 ,source.start_time
                                 ,source.end_time
                                 ,NULL
                                 ,source.data_watermark_value
                                 ,NULL
                                 ,NULL
                                 ,NULL
                                 ,source.commit_time
                                 );  
              """) 
                              
first_run = spark.sql(f"""
                       select count(1) from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}'
                       """).collect()[0]['count(1)']

#scene_id  1:第一次跑  2:源表主表重新初始化 2:源表副表重新初始化 4:源表没有新增 5:正常
(scene_id1, version1, max_version1, last_commit_time1) = get_source_table_version_info(tgt_objectname, src_objectname1)
(scene_id2, version2, max_version2, last_commit_time2) = get_source_table_version_info(tgt_objectname, src_objectname2)

print("scene_id1 = %s" %scene_id1)
print("version1 = %s" %version1)
print("max_version1 = %s" %max_version1)

print("scene_id2 = %s" %scene_id2)
print("version2 = %s" %version2)
print("max_version2 = %s" %max_version2)

if scene_id1 == 3 or scene_id1 == 5:
    source_incr1 = f"""
                       select id, _change_type, _commit_timestamp from table_changes('{src_objectname1}', {version1}, {max_version1}) 
                       where _change_type <> 'update_preimage' 
                       and _commit_timestamp > cast('{last_commit_time1}' as timestamp)
                    """
else:
    source_incr1 = f"""
                       select null as id, null as _change_type, null as _commit_timestamp
                    """
                              
if scene_id2 == 3 or scene_id2 == 5:
    source_incr2 = f"""
                       select quoteid as id, _change_type, _commit_timestamp from table_changes('{src_objectname2}', {version2}, {max_version2}) 
                       where _change_type <> 'update_preimage' 
                       and _commit_timestamp > cast('{last_commit_time2}' as timestamp)
                    """
else:
    source_incr2 = f"""
                       select null as id, null as _change_type, null as _commit_timestamp
                    """                             

#第一次执行或者主表中有一个被初始化
if first_run == 0 or main_tab_no_data_num.count(1) >= 1:
    print('Full capacity run number !')
    sql_options = ''
    change_type_logic = f"""
                             '' as change_type,
                         """
else:
    if all_tab_no_data_num.count(1) == len(main_flg):
        sys.exit()
    else:
        print('Incremental run number !')
        #获取增量数据
        spark.sql(f"""
                        create or replace temp view id_all
                        as
                        select 
                            id
                            ,_change_type as change_type
                            ,row_number() over (partition by id order by _change_type, _commit_timestamp desc) as rn
                        from(
                        {source_incr1}
                        union
                        {source_incr2}
                        ) t
                        where _change_type is not null
                    """
                 )
                              
        sql_options = f"""
                           inner join id_all c 
                           on a.id = c.id
                           and c.rn = 1
                       """
        change_type_logic = f"""
                                 case when a.__src_record_del_status = 'D' or b.__src_record_del_status = 'D' then 'delete'
                                 else c.change_type end  as change_type,
                             """
        

#正式获取数据
sql_txt2 = f"""
                create or replace temp view tmp_lbp_quotation
                as
                select 
                    t.quotation_no,
                    t.quotation_ln_no,
                    t.product_no,
                    t.quotation_qty,
                    t.pre_voucher_price_amt,
                    t.ordering_price_amt,
                    t.dw_source_id,
                    t.dw_insert_dt,
                    t.dw_update_dt,
                    t.dw_del_flg,
                    t.change_type
	            from (
                    select
                        a.quotenumber     as quotation_no,
                        b.itemno          as quotation_ln_no,
                        b.productnumber   as product_no,
                        b.quantity        as quotation_qty,
                        0                 as pre_voucher_price_amt,--no metadata
                        0                 as ordering_price_amt,--no metadata
                        a.__dl_source_id  as dw_source_id,
                        now()             as dw_insert_dt,
                        now()             as dw_update_dt,
                        ''                as dw_del_flg,
                        {change_type_logic}
                        row_number() over (partition by a.quotenumber order by a.__src_record_del_status desc, a.__dl_update_dt desc ) as rn
                    from prep_ms_row.lbp_quotation a
                    {sql_options}
                    inner join prep_ms_row.lbp_quoteline b
                       on a.id = b.quoteid
                      and b.itemno is not null
                    where a.quotenumber is not null
                     ) t 
	            where  rn = 1
                ;

"""

sql_txt3 = f"""
                   create or replace temp view tmp_ln_quotation
                   as 
                   select 
                       *
                       ,'' as change_type
                   from (
                       select 
                           quotation_no,
                           quotation_ln_no,
                           product_no,
                           quotation_qty,
                           pre_voucher_price_amt,
                           ordering_price_amt,
                           dw_source_id,
                           dw_insert_dt,
                           dw_update_dt,
                           dw_del_flg
                       from tmp_lbp_quotation
                       where change_type <> 'delete'
                       except
                       select 
                           quotation_no,
                           quotation_ln_no,
                           product_no,
                           quotation_qty,
                           pre_voucher_price_amt,
                           ordering_price_amt,
                           dw_source_id,
                           dw_insert_dt,
                           dw_update_dt,
                           dw_del_flg
                       from dwd_ms.ln_quotation
                   ) t
                   union
                       select 
                           quotation_no,
                           quotation_ln_no,
                           product_no,
                           quotation_qty,
                           pre_voucher_price_amt,
                           ordering_price_amt,
                           dw_source_id,
                           dw_insert_dt,
                           dw_update_dt,
                           dw_del_flg,
                           change_type
                       from tmp_lbp_quotation
                       where change_type = 'delete'
                   ;
               """;

#merger into 
sql_txt4 = f"""
                   merge into dwd_ms.ln_quotation as target
                   using tmp_ln_quotation as source
                   on target.quotation_no = source.quotation_no
                   when matched and source.change_type = 'delete' then
                     delete
                   when matched and source.change_type = '' then
                     update set target.quotation_ln_no        = source.quotation_ln_no
                                ,target.product_no            = source.product_no
                                ,target.quotation_qty         = source.quotation_qty
                                ,target.pre_voucher_price_amt = source.pre_voucher_price_amt
                                ,target.ordering_price_amt    = source.ordering_price_amt
                                ,target.dw_source_id          = source.dw_source_id
                                ,target.dw_insert_dt          = source.dw_insert_dt
                                ,target.dw_update_dt          = source.dw_update_dt
                                ,target.dw_del_flg            = source.dw_del_flg
                   when not matched and source.change_type = ''
                     then insert(target.quotation_no,target.quotation_ln_no,target.product_no,target.quotation_qty,target.pre_voucher_price_amt,target.ordering_price_amt,target.dw_source_id,target.dw_insert_dt,target.dw_update_dt,target.dw_del_flg) 
                          values(source.quotation_no,source.quotation_ln_no,source.product_no,source.quotation_qty,source.pre_voucher_price_amt,source.ordering_price_amt,source.dw_source_id,source.dw_insert_dt,source.dw_update_dt,source.dw_del_flg)
                   ;
               """;

spark.sql(sql_txt2)
spark.sql(sql_txt3)
spark.sql(sql_txt4)

#update edw_loading_control list
end_time = datetime.datetime.now();
                              
update_control_table(tgt_objectname, src_objectname1, version1, max_version1)
update_control_table(tgt_objectname, src_objectname2, version2, max_version2)

In [ ]:
%sql
select * from dwd_ms.ln_quotation;

In [ ]:
%sql
select * from id_all;

In [ ]:
%sql
select * from dwd_ms.edw_loading_control 
--where tgt_objectname = 'dwd_ms.ln_quotation'
order by tgt_objectname
;

In [ ]:
%sql
delete from dwd_ms.edw_loading_control 
where tgt_objectname = 'dwd_ms.ln_quotation'
;

In [ ]:
%sql
select * from table_changes('dwd_ms.edw_loading_control' , 100);